In [1]:
import sys
import os
sys.path.append(os.path.join('/home','kmendoza','Western_US_Mesh_Dev'))
sys.path.append(os.path.join('/home','kmendoza','egi_utils','egi_utils'))
import meshfracs
import timeit
import numpy as np

In [2]:
from meshfracs.aspect import _patch_is_rectangular
x = np.linspace(0,1,num=2)
# square check
square_x, square_y = np.meshgrid(x,x,indexing='ij')
assert _patch_is_rectangular(square_x, square_y), 'failed square check'

# rect 1 check
square_x, square_y = np.meshgrid(x,x*5,indexing='ij')
assert _patch_is_rectangular(square_x, square_y), 'failed y rect check'

# rect 2 check
square_x, square_y = np.meshgrid(x*5,x,indexing='ij')
assert _patch_is_rectangular(square_x, square_y), 'failed x rect check'

# non_rect_check
xp = np.copy(x)
square_x, square_y = np.meshgrid(x,x,indexing='ij')
square_x[0,0]= square_x[0,0]-0.1
assert not _patch_is_rectangular(square_x, square_y), 'failed not a rect check'

In [3]:
from meshfracs.aspect import _aspect_ratio_exceeds_criteria, _get_max_coordinate_diff

xdiff = _get_max_coordinate_diff(square_x)
ydiff = _get_max_coordinate_diff(square_y)
assert _aspect_ratio_exceeds_criteria(xdiff,ydiff)==0, 'failed nominal check'

xdiff = _get_max_coordinate_diff(square_x*5)
ydiff = _get_max_coordinate_diff(square_y)
assert _aspect_ratio_exceeds_criteria(xdiff,ydiff)==2, 'failed aspect exceeds xdiff check'

xdiff = _get_max_coordinate_diff(square_x)
ydiff = _get_max_coordinate_diff(square_y*5)
assert _aspect_ratio_exceeds_criteria(xdiff,ydiff)==1, 'failed aspect exceeds ydiff check'

In [4]:
from meshfracs.aspect import _correct_square_aspect_ratio

new_x, new_y = _correct_square_aspect_ratio(square_x*5,square_y)
xdiff = _get_max_coordinate_diff(new_x)
ydiff = _get_max_coordinate_diff(new_y)
assert xdiff/ydiff==4.0, 'failed xdiff is too large check'

new_x, new_y = _correct_square_aspect_ratio(square_x,square_y*5)
xdiff = _get_max_coordinate_diff(new_x)
ydiff = _get_max_coordinate_diff(new_y)
assert xdiff/ydiff==1/4, 'failed ydiff is too large check'

In [5]:
from meshfracs.aspect import _quadrilateral_aspect_ratio_correction


square_x, square_y = np.meshgrid(x,x,indexing='ij')
square_x[1,0]+=0.1;
square_y*=9

new_x, new_y = _quadrilateral_aspect_ratio_correction(square_x,square_y)
alt_x, alt_y = _quadrilateral_aspect_ratio_correction(new_x,new_y)
assert np.all(alt_x==new_x) & np.all(alt_y==new_y), 'irregular correction did not work for x direction'

square_x, square_y = np.meshgrid(x,x,indexing='ij')
square_y[0,1]+=0.1;
square_x*=9

new_x, new_y = _quadrilateral_aspect_ratio_correction(square_x,square_y)
alt_x, alt_y = _quadrilateral_aspect_ratio_correction(new_x,new_y)
assert np.all(alt_x==new_x) & np.all(alt_y==new_y), 'irregular correction did not work for y direction'

array([0., 9.])